<a href="https://colab.research.google.com/github/abdelaziz2003vvb/Speech-Recognition-using-DL-MFCC/blob/main/speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model Architecture (Deep Learning)**

We’ll use a Convolutional Neural Network (CNN) on Mel Spectrograms or MFCC features extracted from audio

**pipeline**



```
# Audio file (.wav)
      ↓
Feature extraction (MFCCs / Spectrogram)
      ↓
Deep Learning Model (CNN or LSTM)
      ↓
Softmax Layer → predicts speaker ID
```








# **Step 1: Install dependencies**

# **Step 2: Load and preprocess audio**

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/dataset_speech/archive_dataset.zip'
extract_path = '/content/drive/MyDrive/dataset_speech/'

# Unzip only if not already extracted
if not os.path.exists(os.path.join(extract_path, 'dataset')):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ Dataset extracted successfully!")
else:
    print("✅ Dataset already extracted!")

dataset_path = os.path.join(extract_path, 'dataset')


✅ Dataset extracted successfully!


In [5]:
import os

extract_path = '/content/drive/MyDrive/dataset_speech/'
print(os.listdir(extract_path))


['archive_dataset.zip', '16000_pcm_speeches']


In [6]:
dataset_path = '/content/drive/MyDrive/dataset_speech/16000_pcm_speeches'


In [7]:
import librosa
import numpy as np
import soundfile as sf

def extract_features(file_path, max_pad_len=200):
    try:
        # --- Try normal loading ---
        audio, sample_rate = librosa.load(file_path, sr=None, mono=True)
    except Exception as e:
        print(f"⚠️ Librosa failed for {file_path} → trying raw PCM conversion")

        # --- Fallback: convert raw PCM to WAV ---
        import subprocess
        temp_wav = "/content/temp.wav"
        try:
            subprocess.run([
                "ffmpeg", "-y",
                "-f", "s16le",
                "-ar", "16000",
                "-ac", "1",
                "-i", file_path,
                temp_wav
            ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            audio, sample_rate = librosa.load(temp_wav, sr=None)
        except Exception as e2:
            print(f"❌ Failed to process {file_path}: {e2}")
            return None

    # --- Extract MFCCs ---
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    if pad_width > 0:
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :max_pad_len]
    return mfccs

In [8]:
# ✅ Step 5: Load dataset
data, labels = [], []
for speaker in os.listdir(dataset_path):
    speaker_path = os.path.join(dataset_path, speaker)
    if not os.path.isdir(speaker_path):
        continue
    for file in os.listdir(speaker_path):
        if file.endswith(".wav"):
            file_path = os.path.join(speaker_path, file)
            features = extract_features(file_path)
            if features is not None:
                data.append(features)
                labels.append(speaker)

X = np.array(data)
y = np.array(labels)
print("✅ Dataset loaded:", X.shape, len(y), "samples")


✅ Dataset loaded: (7507, 40, 200) 7507 samples


In [9]:
# ✅ Step 6: Prepare data
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

le = LabelEncoder()
y_encoded = to_categorical(le.fit_transform(y))

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]


In [10]:
# ✅ Step 7: CNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(40, 200, 1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_encoded.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 38, 198, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 19, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 17, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 48, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 24576)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,145,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,165,575 (12.08 MB)

 Trainable params: 3,165,575 (12.08 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# ✅ Step 8: Train
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 74s 383ms/step - accuracy: 0.6995 - loss: 1.3769 - val_accuracy: 0.9867 - val_loss: 0.0589
Epoch 2/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 75s 348ms/step - accuracy: 0.9718 - loss: 0.1016 - val_accuracy: 0.9880 - val_loss: 0.0379
Epoch 3/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 66s 349ms/step - accuracy: 0.9908 - loss: 0.0386 - val_accuracy: 0.9880 - val_loss: 0.0298
Epoch 4/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 84s 364ms/step - accuracy: 0.9897 - loss: 0.0352 - val_accuracy: 0.9840 - val_loss: 0.0491
Epoch 5/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 79s 349ms/step - accuracy: 0.9846 - loss: 0.0381 - val_accuracy: 0.9867 - val_loss: 0.0438


In [12]:
# ✅ Step 9: Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"🎯 Test Accuracy: {acc*100:.2f}%")


47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 105ms/step - accuracy: 0.9895 - loss: 0.0502
🎯 Test Accuracy: 98.67%


In [13]:
# ✅ Step 10: Test prediction on a sample
test_file = "/content/drive/MyDrive/dataset_speech/16000_pcm_speeches/Julia_Gillard/0.wav"

feature = extract_features(test_file)
if feature is not None:
    prediction = model.predict(feature[np.newaxis, ..., np.newaxis])
    predicted_speaker = le.inverse_transform([np.argmax(prediction)])
    print("🔊 Recognized Speaker:", predicted_speaker[0])
else:
    print("❌ Could not extract features from the test file.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
🔊 Recognized Speaker: Julia_Gillard
